In [2]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The first number")
    b: float = Field(description="The second number")


class CalculatorTool(BaseTool):
    name = "CalculatorTool"
    description = """
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
    args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return a + b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        CalculatorTool(),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)



> Entering new AgentExecutor chain...
Thought: I can use the Sum Calculator tool to find the total cost of all the individual amounts provided by the user.

Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 355.39,
    "b": 924.87
  }
}
```
Observation: 1280.26
Thought:```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 1280.26,
    "b": 721.2
  }
}
```
Observation: 2001.46
Thought:```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 2001.46,
    "b": 1940.29
  }
}
```
Observation: 3941.75
Thought:```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 3941.75,
    "b": 573.63
  }
}
```
Observation: 4515.38
Thought:```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 4515.38,
    "b": 65.72
  }
}
```
Observation: 4581.1
Thought:```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 4581.1,
    "b": 35.00
  }
}
```
Observation: 4616.1
Thought:```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 

{'input': 'Tell me Total Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'I have calculated the total cost of all the individual amounts provided by the user. The final answer is $5273.38.'}